In [17]:
import pickle
import pandas as pd
import numpy as np
import os

In [18]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(color, year, month):
    input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{color}_tripdata_{year}-{month:02d}.parquet'
    df = pd.read_parquet(input_file)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    return df


def load_model(filename):
    with open(filename, 'rb') as f_in:
        dv, model = pickle.load(f_in)
    return dv, model

def apply_model(color, year, month, model_file):
    df = read_data(color, year, month)
    dv, model = load_model(model_file)
    output_file = f'output/{color}/{year:04d}-{month:02d}.parquet'
    output_dir = os.path.dirname(output_file)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)    

    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = model.predict(X_val)
    df['prediction'] = y_pred 

    df_result = df[['ride_id', 'prediction']].copy()
    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )
    return df_result

In [ ]:
filename = 'model.bin'
color  = 'yellow'
year = 2023
month = 3

df_yellow_202303 = apply_model(color, year, month, filename)

c:\Users\sergi\OneDrive\Documentos\MLOps2025\mlops-sergiosm3\04-deployment\venvDeployment\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sergi\OneDrive\Documentos\MLOps2025\mlops-sergiosm3\04-deployment\venvDeployment\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [21]:
df_yellow_202303.prediction.std()

np.float64(6.247489794199401)